<a href="https://colab.research.google.com/github/padariya-tech/ML-Projects/blob/main/linguistic_Assignment2_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import defaultdict

# Define the named entity types
entity_types = ['PER', 'LOC', 'ORG', 'MISC','O']

# Dictionary to store sum of F1 scores for each class
class_f1_sum = defaultdict(float)

# Dictionary to store count of lines for each class
class_count = defaultdict(int)

# Read lines from files
with open("/content/manual_ans.txt", "r") as file:
    manual_ans = [line.strip() for line in file.readlines()]

with open("/content/chatgpt_ans.txt", "r") as file:
    chat_ans = [line.strip() for line in file.readlines()]

def calculate_scores(true_labels, pred_labels):
    classes = set(true_labels).union(set(pred_labels))
    precision = {}
    recall = {}
    f1 = {}
    for cls in classes:
        true_positives = sum([1 for true, pred in zip(true_labels, pred_labels) if true == cls and pred == cls])
        false_positives = sum([1 for true, pred in zip(true_labels, pred_labels) if true != cls and pred == cls])
        false_negatives = sum([1 for true, pred in zip(true_labels, pred_labels) if true == cls and pred != cls])
        precision[cls] = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall[cls] = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1[cls] = 2 * (precision[cls] * recall[cls]) / (precision[cls] + recall[cls]) if precision[cls] + recall[cls] > 0 else 0
    return precision, recall, f1

count = 1
# Iterate over each line
for manual_line, chat_line in zip(manual_ans, chat_ans):
    # Split the lines into individual elements
    manual_elements = manual_line.split()
    chat_elements = chat_line.split()

    # Calculate precision, recall, and F1 score for the current line
    precision, recall, f1_score = calculate_scores(manual_elements, chat_elements)

    # Print the results
    print(f"line number :{count}")
    for cls in precision.keys():
        print(f"Class: {cls}")
        print(f"  Precision: {precision[cls]}")
        print(f"  Recall: {recall[cls]}")
        print(f"  F1 Score: {f1_score[cls]}")
        class_f1_sum[cls] += f1_score[cls]
        class_count[cls] += 1
    print()
    count = count + 1

overall_macro_f1 = {}
for cls in class_f1_sum.keys():
    overall_macro_f1[cls] = class_f1_sum[cls] / class_count[cls] if class_count[cls] > 0 else 0
addval=0
# Print the overall macro F1 scores for each class
for cls in overall_macro_f1.keys():
    print(f"Class: {cls}")
    print(f"  Overall Macro F1 Score: {overall_macro_f1[cls]}")
    addval += overall_macro_f1[cls]


print(f"Sum of Overall Macro F1 Scores: {addval/9}")

line number :1
Class: O
  Precision: 0.9090909090909091
  Recall: 1.0
  F1 Score: 0.9523809523809523
Class: B-LOC
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-MISC
  Precision: 0
  Recall: 0.0
  F1 Score: 0

line number :2
Class: O
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-PER
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-LOC
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: I-PER
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0

line number :3
Class: O
  Precision: 0.8125
  Recall: 1.0
  F1 Score: 0.896551724137931
Class: B-LOC
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-ORG
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-MISC
  Precision: 0
  Recall: 0.0
  F1 Score: 0

line number :4
Class: O
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-LOC
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-PER
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: I-LOC
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0

line number

# **check 25 sentence on model indicBERT**

In [2]:
from transformers import AutoModel, AutoTokenizer ,AutoModelForTokenClassification
import torch
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert',keep_accents=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
label_list=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [4]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [5]:
import json

In [6]:
config = json.load(open("/content/drive/MyDrive/ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("/content/drive/MyDrive/ner_model/config.json","w"))

In [7]:
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/ner_model")

In [8]:
model

AlbertForTokenClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, b

In [9]:
def get_ner(sentence):
    tok_sentence = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        logits = model(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            model.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

        ner_output = []
        for index in range(len(sentence.split(' '))):
            if(index<len(predicted_labels)):
                ner_output.append((sentence.split(' ')[index], predicted_labels[index]))
            else:
                ner_output.append((sentence.split(' ')[index], 'O'))
        return ner_output


In [10]:
sentence = 'આ જીતની સાથે જ હૈદરાબાદની ટીમ પાંચમાં સ્થાન પર છે જ્યારે મુંબઇની ટીમ નેટ રન રેટ સારો હોવાના કારણે ચોથા સ્થાન પર કાયમ છે.'

predicted_labels = get_ner(sentence)
print(predicted_labels)

[('આ', 'O'), ('જીતની', 'O'), ('સાથે', 'O'), ('જ', 'O'), ('હૈદરાબાદની', 'B-LOC'), ('ટીમ', 'O'), ('પાંચમાં', 'O'), ('સ્થાન', 'O'), ('પર', 'O'), ('છે', 'O'), ('જ્યારે', 'O'), ('મુંબઇની', 'B-LOC'), ('ટીમ', 'O'), ('નેટ', 'O'), ('રન', 'O'), ('રેટ', 'O'), ('સારો', 'O'), ('હોવાના', 'O'), ('કારણે', 'O'), ('ચોથા', 'O'), ('સ્થાન', 'O'), ('પર', 'O'), ('કાયમ', 'O'), ('છે.', 'O')]


In [11]:
file_path = '/content/drive/MyDrive/que_25.txt'
predicted_labels_list = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        predicted_labels = get_ner(line.strip())
        predicted_labels_values = [label for _, label in predicted_labels]
        predicted_labels_list.append(predicted_labels_values)

print(predicted_labels_list)


[['O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O'

In [12]:
single_list = [' '.join(sublist) for sublist in predicted_labels_list]

print(single_list)

['O O O O B-LOC O O O O O O B-LOC O O O O O O O O O O O O', 'O O B-PER I-PER O O O O O O', 'O O O O O O O O O O O O O O O O O O O O', 'O O O O O O O O O O O O', 'O O O O O O O O O O O', 'O O O O O O O O O O O O O O O', 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O', 'B-PER I-PER O O O O O O O O O O O O O', 'O O O B-PER I-PER O O O O O O O O O O O O O O O O O O O O O O O O O O O O O', 'B-ORG O B-ORG O O O O O O O O', 'O O O O O O O O O O', 'O O O O O O O O O O O O', 'O O O O O O O O O B-ORG O O O O O O O B-PER I-PER O O O O O', 'B-ORG I-ORG O O O O O O O O O', 'O O O O O O O O O O O', 'O O O O O O B-ORG O O O O O B-ORG B-ORG O O O O O', 'O O O O O O O O O O', 'O O O O O B-PER I-PER O B-PER I-PER O O', 'O O O O O O O O O O O O O O O O O O O', 'O O O O O O O O O', 'O O O O O O O O O O O O O O O O O', 'O O O O O O O O O O O O O O O O', 'O O O O O O O O O', 'O O O O O O O O O O B-ORG O O O O O O O O', 'O O O O O O O O O O O O', 'O']


In [19]:
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import defaultdict
# Define the named entity types
entity_types = ['PER', 'LOC', 'ORG', 'MISC','O']


# Dictionary to store sum of F1 scores for each class
class_f1_sum = defaultdict(float)

# Dictionary to store count of lines for each class
class_count = defaultdict(int)
# Read lines from files


def calculate_scores(true_labels, pred_labels):
    classes = set(true_labels).union(set(pred_labels))
    precision = {}
    recall = {}
    f1_score = {}
    for cls in classes:
        true_positives = sum([1 for true, pred in zip(true_labels, pred_labels) if true == cls and pred == cls])
        false_positives = sum([1 for true, pred in zip(true_labels, pred_labels) if true == cls and pred != cls])
        false_negatives = sum([1 for true, pred in zip(true_labels, pred_labels) if true == cls and pred != cls])
        precision[cls] = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall[cls] = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score[cls] = 2 * (precision[cls] * recall[cls]) / (precision[cls] + recall[cls]) if precision[cls] + recall[cls] > 0 else 0
    return precision, recall, f1_score
count =1;
# Iterate over each line
# print(manual_ans)

for manual_line, chat_line in zip(manual_ans, single_list):
    # Split the lines into individual elements
    manual_elements = manual_line.split()
    chat_elements = chat_line.split()

    # Calculate precision, recall, and F1 score for the current line
    precision, recall, f1_score = calculate_scores(manual_elements, chat_elements)

    # Print the results
    print(f"line number :{count}")
    for cls in precision.keys():
        print(f"Class: {cls}")
        print(f"  Precision: {precision[cls]}")
        print(f"  Recall: {recall[cls]}")
        print(f"  F1 Score: {f1_score[cls]}")
        class_f1_sum[cls] += f1_score[cls]
        class_count[cls] += 1
    print()
    count=count+1
overall_macro_f1 = {}
for cls in class_f1_sum.keys():
    overall_macro_f1[cls] = class_f1_sum[cls] / class_count[cls] if class_count[cls] > 0 else 0

# Print the overall macro F1 scores for each class
addval=0
for cls in overall_macro_f1.keys():
    print(f"Class: {cls}")
    print(f"  Overall Macro F1 Score: {overall_macro_f1[cls]}")
    addval += overall_macro_f1[cls]

print(f"Sum of Overall Macro F1 Scores: {addval/9}")

line number :1
Class: B-MISC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0
Class: O
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-LOC
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0

line number :2
Class: I-PER
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-PER
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-LOC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0
Class: O
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0

line number :3
Class: B-ORG
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0
Class: B-MISC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0
Class: O
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-LOC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0

line number :4
Class: B-PER
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0
Class: B-LOC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0
Class: O
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: I-LOC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0

line number :5
Class: B-MISC
  Precision: 0.0
  Recall: 0.0
  F1 S

# **check 25 sentence on model IndicNER**

In [1]:
from transformers import AutoModel, AutoTokenizer ,AutoModelForTokenClassification
import torch
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/IndicNER',keep_accents=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [2]:
label_list=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [3]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [4]:
import json

In [5]:
config = json.load(open("/content/drive/MyDrive/indic_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("/content/drive/MyDrive/indic_model/config.json","w"))

In [6]:
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/indic_model")

In [7]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [8]:
def get_ner(sentence):
    tok_sentence = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        logits = model(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            model.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

        ner_output = []
        for index in range(len(sentence.split(' '))):
            if(index<len(predicted_labels)):
                ner_output.append((sentence.split(' ')[index], predicted_labels[index]))
            else:
                ner_output.append((sentence.split(' ')[index], 'O'))
        return ner_output


In [9]:
sentence = 'આ જીતની સાથે જ હૈદરાબાદની ટીમ પાંચમાં સ્થાન પર છે જ્યારે મુંબઇની ટીમ નેટ રન રેટ સારો હોવાના કારણે ચોથા સ્થાન પર કાયમ છે.'

predicted_labels = get_ner(sentence)
print(predicted_labels)

[('આ', 'O'), ('જીતની', 'O'), ('સાથે', 'O'), ('જ', 'O'), ('હૈદરાબાદની', 'B-LOC'), ('ટીમ', 'O'), ('પાંચમાં', 'O'), ('સ્થાન', 'O'), ('પર', 'O'), ('છે', 'O'), ('જ્યારે', 'O'), ('મુંબઇની', 'B-LOC'), ('ટીમ', 'O'), ('નેટ', 'O'), ('રન', 'O'), ('રેટ', 'O'), ('સારો', 'O'), ('હોવાના', 'O'), ('કારણે', 'O'), ('ચોથા', 'O'), ('સ્થાન', 'O'), ('પર', 'O'), ('કાયમ', 'O'), ('છે.', 'O')]


In [10]:
file_path = '/content/drive/MyDrive/que_25.txt'
predicted_labels_list = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        predicted_labels = get_ner(line.strip())
        predicted_labels_values = [label for _, label in predicted_labels]
        predicted_labels_list.append(predicted_labels_values)

print(predicted_labels_list)


[['O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'O', 'B-ORG', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O'

In [11]:
single_list = [' '.join(sublist) for sublist in predicted_labels_list]

print(single_list)

['O O O O B-LOC O O O O O O B-LOC O O O O O O O O O O O O', 'B-LOC O B-PER I-PER O O O O O O', 'B-LOC O B-ORG B-ORG O O O O O O O O O O O O O O O O', 'B-PER O O O O O O O O O B-LOC O', 'O O O O O O O O O O O', 'B-PER O O O O O O O O O O O O O O', 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O', 'B-PER I-PER O O O O O O O O O O O O O', 'O O O B-PER I-PER O O O B-ORG I-ORG O O O O O O O O O O O O O O O O O O O O O O O O', 'B-LOC O O O O O O O O O O', 'O O O O O O O O O O', 'O O O O O O O O O O O O', 'O O O O O O O O O B-ORG I-ORG I-ORG I-ORG O O O O B-PER I-PER O O O O O', 'B-LOC O O O O O O O B-PER O O', 'O O O O O O O O O O O', 'O O O B-ORG I-ORG I-ORG B-ORG O O O O O O B-ORG O O O O O', 'O O O O O O O O O O', 'O O O O O B-PER I-PER O B-PER I-PER O O', 'O O O O O O O O O O O O O O O O O O O', 'O O O O O O O O O', 'B-LOC O I-ORG I-ORG O O O O O O O O O O O O O', 'O O O O O O O O O O O O O O O O', 'O O O O O O O O O', 'O O O O O O O O O O O O O B-ORG O O O O O', 'O 

In [15]:
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import defaultdict
# Define the named entity types
entity_types = ['PER', 'LOC', 'ORG', 'MISC','O']


# Dictionary to store sum of F1 scores for each class
class_f1_sum = defaultdict(float)

# Dictionary to store count of lines for each class
class_count = defaultdict(int)
# Read lines from files


def calculate_scores(true_labels, pred_labels):
    classes = set(true_labels).union(set(pred_labels))
    precision = {}
    recall = {}
    f1_score = {}
    for cls in classes:
        true_positives = sum([1 for true, pred in zip(true_labels, pred_labels) if true == cls and pred == cls])
        false_positives = sum([1 for true, pred in zip(true_labels, pred_labels) if true == cls and pred != cls])
        false_negatives = sum([1 for true, pred in zip(true_labels, pred_labels) if true == cls and pred != cls])
        precision[cls] = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall[cls] = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score[cls] = 2 * (precision[cls] * recall[cls]) / (precision[cls] + recall[cls]) if precision[cls] + recall[cls] > 0 else 0
    return precision, recall, f1_score
count =1;
# Iterate over each line
# print(manual_ans)

for manual_line, chat_line in zip(manual_ans, single_list):
    # Split the lines into individual elements
    manual_elements = manual_line.split()
    chat_elements = chat_line.split()

    # Calculate precision, recall, and F1 score for the current line
    precision, recall, f1_score = calculate_scores(manual_elements, chat_elements)

    # Print the results
    print(f"line number :{count}")
    for cls in precision.keys():
        print(f"Class: {cls}")
        print(f"  Precision: {precision[cls]}")
        print(f"  Recall: {recall[cls]}")
        print(f"  F1 Score: {f1_score[cls]}")
        class_f1_sum[cls] += f1_score[cls]
        class_count[cls] += 1
    print()
    count=count+1
overall_macro_f1 = {}
for cls in class_f1_sum.keys():
    overall_macro_f1[cls] = class_f1_sum[cls] / class_count[cls] if class_count[cls] > 0 else 0

# Print the overall macro F1 scores for each class
addval=0
for cls in overall_macro_f1.keys():
    print(f"Class: {cls}")
    print(f"  Overall Macro F1 Score: {overall_macro_f1[cls]}")
    addval += overall_macro_f1[cls]

print(f"Sum of Overall Macro F1 Scores: {addval/9}")

line number :1
Class: O
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-LOC
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-MISC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0

line number :2
Class: O
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-PER
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-LOC
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: I-PER
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0

line number :3
Class: O
  Precision: 0.9230769230769231
  Recall: 0.9230769230769231
  F1 Score: 0.9230769230769231
Class: B-LOC
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-ORG
  Precision: 0.5
  Recall: 0.5
  F1 Score: 0.5
Class: B-MISC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0

line number :4
Class: O
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: B-LOC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0
Class: B-PER
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class: I-LOC
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0

line number :